# Databricks Serverless Module Loading Solutions

This notebook demonstrates how to resolve ImportError issues with custom modules in Databricks Serverless notebooks, specifically for the `hardware_store` module.

**Problem**: `ImportError: Failed to import hardware_store.tools.find_product_details_by_description_tool: No module named 'hardware_store'`

**Context**: Works on regular Databricks compute but fails on serverless notebooks.

## 1. Verify Current Working Directory and Path

Check the current working directory and sys.path to understand the environment differences between serverless and regular compute.

In [ ]:
import os
import sys
from pathlib import Path

print("Current working directory:")
print(os.getcwd())
print("\nCurrent sys.path:")
for i, path in enumerate(sys.path):
    print(f"  {i}: {path}")

print("\nChecking if src directory exists:")
src_paths = ['./src', 'src', '../src']
for path in src_paths:
    if os.path.exists(path):
        print(f"✓ Found: {os.path.abspath(path)}")
        # List contents
        contents = os.listdir(path)
        print(f"  Contents: {contents}")
    else:
        print(f"✗ Not found: {path}")

## 2. Check Module Structure and __init__.py Files

Verify that the hardware_store module has proper __init__.py files and correct directory structure for Python package recognition.

In [ ]:
def check_module_structure(base_path):
    """Check if the module has proper Python package structure"""
    print(f"Checking module structure in: {base_path}")
    
    # Check for hardware_store directory
    hw_store_path = os.path.join(base_path, 'hardware_store')
    if os.path.exists(hw_store_path):
        print(f"✓ Found hardware_store directory: {hw_store_path}")
        
        # Check for __init__.py
        init_file = os.path.join(hw_store_path, '__init__.py')
        if os.path.exists(init_file):
            print(f"✓ Found __init__.py: {init_file}")
        else:
            print(f"✗ Missing __init__.py: {init_file}")
            # Create it if missing
            try:
                with open(init_file, 'w') as f:
                    f.write('# Hardware store package\n')
                print(f"✓ Created __init__.py: {init_file}")
            except Exception as e:
                print(f"✗ Failed to create __init__.py: {e}")
        
        # Check tools subdirectory
        tools_path = os.path.join(hw_store_path, 'tools')
        if os.path.exists(tools_path):
            print(f"✓ Found tools directory: {tools_path}")
            
            # Check for tools __init__.py
            tools_init = os.path.join(tools_path, '__init__.py')
            if os.path.exists(tools_init):
                print(f"✓ Found tools/__init__.py: {tools_init}")
            else:
                print(f"✗ Missing tools/__init__.py: {tools_init}")
                try:
                    with open(tools_init, 'w') as f:
                        f.write('# Hardware store tools package\n')
                    print(f"✓ Created tools/__init__.py: {tools_init}")
                except Exception as e:
                    print(f"✗ Failed to create tools/__init__.py: {e}")
            
            # List all Python files in tools
            tools_files = [f for f in os.listdir(tools_path) if f.endswith('.py')]
            print(f"  Python files in tools: {tools_files}")
        else:
            print(f"✗ Missing tools directory: {tools_path}")
    else:
        print(f"✗ hardware_store directory not found: {hw_store_path}")

# Check in common locations
for path in ['./src', 'src', '.', '../src']:
    if os.path.exists(path):
        check_module_structure(path)
        print()

## 3. Alternative Import Methods for Serverless

Implement different approaches to import modules in serverless environment, including absolute imports and sys.path modifications.

In [ ]:
# Method 1: Enhanced sys.path modification
def add_source_paths():
    """Add common source paths to sys.path"""
    import sys
    import os
    
    potential_paths = ['./src', 'src', '.', '../src']
    added_paths = []
    
    for path in potential_paths:
        if os.path.exists(path):
            abs_path = os.path.abspath(path)
            if abs_path not in sys.path:
                sys.path.insert(0, abs_path)
                added_paths.append(abs_path)
                print(f"Added to sys.path: {abs_path}")
    
    return added_paths

# Method 2: Invalidate import cache
def refresh_import_cache():
    """Clear import cache to pick up new paths"""
    import importlib
    importlib.invalidate_caches()
    print("Import cache invalidated")

# Method 3: Enhanced load_function (from dao_ai.utils)
def enhanced_load_function(function_name):
    """Load function with enhanced path resolution"""
    from dao_ai.utils import load_function
    return load_function(function_name)

# Try the enhanced methods
print("Method 1: Adding source paths")
added_paths = add_source_paths()

print("\nMethod 2: Refreshing import cache")
refresh_import_cache()

print("\nMethod 3: Testing enhanced load_function")
try:
    func = enhanced_load_function('hardware_store.tools.find_product_details_by_description_tool')
    print("✓ Successfully loaded hardware_store.tools.find_product_details_by_description_tool")
    print(f"Function: {func}")
except Exception as e:
    print(f"✗ Failed to load: {e}")

# Method 4: Direct import test
print("\nMethod 4: Direct import test")
try:
    import hardware_store.tools.find_product_details_by_description_tool
    print("✓ Direct import successful")
except Exception as e:
    print(f"✗ Direct import failed: {e}")

## 4. Install Package in Development Mode

Use pip install -e to install the local package in development mode, making it available system-wide in the serverless environment.

In [ ]:
# Method 5: Install in development mode
import subprocess
import sys

def install_local_package():
    """Install the current project in development mode"""
    try:
        # Find the root directory with setup.py or pyproject.toml
        current_dir = os.getcwd()
        root_candidates = ['.', '..', '../..']
        
        for candidate in root_candidates:
            setup_py = os.path.join(candidate, 'setup.py')
            pyproject = os.path.join(candidate, 'pyproject.toml')
            
            if os.path.exists(setup_py) or os.path.exists(pyproject):
                print(f"Found package root: {os.path.abspath(candidate)}")
                
                # Install in development mode
                cmd = [sys.executable, '-m', 'pip', 'install', '-e', candidate]
                result = subprocess.run(cmd, capture_output=True, text=True)
                
                if result.returncode == 0:
                    print("✓ Successfully installed package in development mode")
                    print(result.stdout)
                    return True
                else:
                    print(f"✗ Failed to install: {result.stderr}")
                    return False
        
        print("✗ No setup.py or pyproject.toml found")
        return False
    except Exception as e:
        print(f"✗ Error during installation: {e}")
        return False

# Only try this if we're in a development environment
if any(os.path.exists(f) for f in ['setup.py', 'pyproject.toml', '../setup.py', '../pyproject.toml']):
    print("Attempting to install package in development mode...")
    install_local_package()
else:
    print("No setup.py or pyproject.toml found - skipping development install")

## 5. Use Databricks File System (DBFS) for Module Loading

Upload the module to DBFS and modify the import strategy to load from the distributed file system.

In [ ]:
# Method 6: DBFS-based module loading (for Databricks environments)
def setup_dbfs_imports():
    """Setup imports from DBFS in Databricks environment"""
    try:
        # Check if we're in Databricks
        if 'DATABRICKS_RUNTIME_VERSION' in os.environ:
            print("Detected Databricks environment")
            
            # Common DBFS paths where modules might be stored
            dbfs_paths = [
                '/dbfs/FileStore/shared_uploads/',
                '/dbfs/mnt/workspace/',
                '/dbfs/tmp/',
            ]
            
            for dbfs_path in dbfs_paths:
                if os.path.exists(dbfs_path):
                    print(f"Found DBFS path: {dbfs_path}")
                    # Look for src directory
                    src_path = os.path.join(dbfs_path, 'src')
                    if os.path.exists(src_path):
                        print(f"Found src in DBFS: {src_path}")
                        if src_path not in sys.path:
                            sys.path.insert(0, src_path)
                            print(f"Added DBFS src to sys.path: {src_path}")
                            return True
            
            print("No src directory found in DBFS paths")
            return False
        else:
            print("Not in Databricks environment - skipping DBFS setup")
            return False
    except Exception as e:
        print(f"Error setting up DBFS imports: {e}")
        return False

# Try DBFS setup
setup_dbfs_imports()

## 6. Workspace Files Integration

Utilize Databricks Workspace Files feature to manage and import custom modules in serverless notebooks.

In [ ]:
# Method 7: Workspace Files integration
def setup_workspace_files():
    """Setup imports from Databricks Workspace Files"""
    try:
        # Check for workspace files paths
        workspace_paths = [
            '/Workspace/Users/',
            '/Workspace/Shared/',
            '/Workspace/Repos/',
        ]
        
        for base_path in workspace_paths:
            if os.path.exists(base_path):
                print(f"Found workspace path: {base_path}")
                # Look for our project directory
                for item in os.listdir(base_path):
                    item_path = os.path.join(base_path, item)
                    if os.path.isdir(item_path):
                        src_path = os.path.join(item_path, 'src')
                        if os.path.exists(src_path):
                            print(f"Found src in workspace: {src_path}")
                            if src_path not in sys.path:
                                sys.path.insert(0, src_path)
                                print(f"Added workspace src to sys.path: {src_path}")
                                return True
        
        print("No src directory found in workspace paths")
        return False
    except Exception as e:
        print(f"Error setting up workspace files: {e}")
        return False

# Try workspace files setup
setup_workspace_files()

## 7. Testing Import Solutions

Test each solution method to verify successful import of the hardware_store.tools.find_product_details_by_description_tool module.

In [ ]:
# Final comprehensive test
def test_all_import_methods():
    """Test all import methods to find working solution"""
    test_results = {}
    
    # Test 1: Direct import
    try:
        import hardware_store.tools.find_product_details_by_description_tool
        test_results['direct_import'] = '✓ Success'
    except Exception as e:
        test_results['direct_import'] = f'✗ Failed: {str(e)[:50]}...'
    
    # Test 2: Enhanced load_function
    try:
        from dao_ai.utils import load_function
        func = load_function('hardware_store.tools.find_product_details_by_description_tool')
        test_results['enhanced_load_function'] = '✓ Success'
    except Exception as e:
        test_results['enhanced_load_function'] = f'✗ Failed: {str(e)[:50]}...'
    
    # Test 3: importlib with path manipulation
    try:
        import importlib
        importlib.invalidate_caches()
        module = importlib.import_module('hardware_store.tools.find_product_details_by_description_tool')
        test_results['importlib_method'] = '✓ Success'
    except Exception as e:
        test_results['importlib_method'] = f'✗ Failed: {str(e)[:50]}...'
    
    # Test 4: Try importing other dao_ai modules to verify base setup
    try:
        from dao_ai.config import AppConfig
        test_results['dao_ai_import'] = '✓ Success'
    except Exception as e:
        test_results['dao_ai_import'] = f'✗ Failed: {str(e)[:50]}...'
    
    return test_results

# Run comprehensive test
print("Running comprehensive import tests...\n")
results = test_all_import_methods()

for method, result in results.items():
    print(f"{method:25}: {result}")

# Summary and recommendations
print("\n" + "="*60)
print("SUMMARY AND RECOMMENDATIONS")
print("="*60)

successful_methods = [method for method, result in results.items() if result.startswith('✓')]

if successful_methods:
    print(f"\n✓ Working methods: {', '.join(successful_methods)}")
    print("\nRecommendations:")
    print("1. Use the enhanced load_function from dao_ai.utils - it handles path issues automatically")
    print("2. Ensure all Python packages have __init__.py files")
    print("3. For Databricks Serverless, consider using Workspace Files or DBFS for module storage")
else:
    print("\n✗ No methods were successful. Troubleshooting steps:")
    print("1. Verify the hardware_store module exists in the src directory")
    print("2. Check that all directories have __init__.py files")
    print("3. Ensure the module structure matches the import path")
    print("4. Consider uploading the module to DBFS or Workspace Files")

# Show current sys.path for debugging
print("\nCurrent sys.path:")
for i, path in enumerate(sys.path[:10]):  # Show first 10 paths
    print(f"  {i}: {path}")
if len(sys.path) > 10:
    print(f"  ... and {len(sys.path) - 10} more paths")